In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import math

In [2]:
import cv20_lab1_part3_utils as p3
import CV20_lab1_Part2
from CV20_lab1_Part2 import EdgeDetectors as edgeDet
from CV20_lab1_Part2 import BlobDetectors as blobDet



### 3.1 Ταίριασμα Εικόνων υπό Περιστροφή και Αλλαγή Κλίμακας
#### Corner Matching

In [3]:
# Here is a lambda which acts as a wrapper for detector function, e.g. harrisDetector.
# The detector arguments are, in order: image, sigma, rho, k, threshold.
detect_fun = np.array([[],''])
desc_fun = np.array([[],''])
detect_fun[0] = [lambda I: edgeDet.Harris_Stephens(I, sigma=2.5, rho=2.5, k=0.05, theta=0.005, save_png=False),'Corner Detector']
detect_fun[1] = [lambda I: edgeDet.Harris_Laplace(I, sigma=2, rho=2.5, k=0.005, theta=0.006, s=1.5, N=5, save_png=True),'Multi-Corner Detector']
 # You can use either of the following lines to extract features (HOG/SURF).
desc_fun[0] = [lambda I, kp: p3.featuresSURF(I,kp),'SURF']
desc_fun[1] = [lambda I, kp: p3.featuresHOG(I,kp),'HOG']

for i in range(2):
    for j in range(2):
        avg_scale_errors,avg_theta_errors = p3.matching_evaluation(detect_fun[i][0], desc_fun[j][0])
        print(detect_fun[i][1])
        print(desc_fun[j][1])
        print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
        print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
        print('%---------------------------------%')





Corner Detector
SURF
Avg. Scale Error for Image 1: 0.066
Avg. Theta Error for Image 1: 1.625
%---------------------------------%
Corner Detector
HOG
Avg. Scale Error for Image 1: 0.507
Avg. Theta Error for Image 1: 24.627
%---------------------------------%
Multi-Corner Detector
SURF
Avg. Scale Error for Image 1: 0.201
Avg. Theta Error for Image 1: 10.730
%---------------------------------%
Multi-Corner Detector
HOG
Avg. Scale Error for Image 1: 0.233
Avg. Theta Error for Image 1: 25.009
%---------------------------------%


In [16]:
detector_funcsMB = np.array([[],[],''])
detector_funcsMB[0] = [lambda I: blobDet.SingleScale_Blobs(I, sigma=2, theta=0.006, approx=False, save_png=True),'Blob Detector']
detector_funcsMB[1] = [lambda I: blobDet.MultiScale_Blobs(I, sigma=2, theta=0.05, s=1.5, N=4, approx=False, save_png=True),'Multi Blob Detector']
detector_funcsMB[2] = [lambda I: blobDet.MultiScale_Blobs(I, sigma=2, theta=0.1, s=2, N=4, approx=True, save_png=True),'Multi Box Blob Detector']

for i in range(3):
    for j in range(2):
        avg_scale_errors,avg_theta_errors = p3.matching_evaluation(detector_funcsMB[i][0], desc_fun[j][0])
        print(detector_funcsMB[i][1])
        print(desc_fun[j][1])
        print('Avg. Scale Error for Image 1: {:.3f}'.format(avg_scale_errors[0]))
        print('Avg. Theta Error for Image 1: {:.3f}'.format(avg_theta_errors[0]))
        print('%---------------------------------%')




Blob Detector
SURF
Avg. Scale Error for Image 1: 0.036
Avg. Theta Error for Image 1: 2.655
%---------------------------------%
Blob Detector
HOG
Avg. Scale Error for Image 1: 0.106
Avg. Theta Error for Image 1: 7.254
%---------------------------------%
Multi Blob Detector
SURF
Avg. Scale Error for Image 1: 0.002
Avg. Theta Error for Image 1: 0.077
%---------------------------------%
Multi Blob Detector
HOG
Avg. Scale Error for Image 1: 0.134
Avg. Theta Error for Image 1: 12.093
%---------------------------------%
Multi Box Blob Detector
SURF
Avg. Scale Error for Image 1: 0.004
Avg. Theta Error for Image 1: 0.178
%---------------------------------%
Multi Box Blob Detector
HOG
Avg. Scale Error for Image 1: 0.099
Avg. Theta Error for Image 1: 12.109
%---------------------------------%


### 3.2 Κατηγοροποίηση Εικόνων


In [17]:
import OurBoVW as OurBagofWords

Ακολουθούν τα ερωτήματα 3.2.1 - 3.2.4 υλοποιώντας για το ερώτημα 3.2.3 τη δική μας BoVW συνάρτηση

### 3.2.1 - 3.2.4 Create Our Bag of Visual Words
Δημιουργούμε τη δική μας BoVW χρησιμοποιώντας για τον αλγόριθμο kmeans 600 κέντρα.


In [4]:
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn import preprocessing

def OurBagVW(data_train, data_test):
    k = 600
    kmeans = KMeans(k) # n_clusters = k 
    # concatenate all features into a global array of features
    all_features = np.concatenate(data_train)
    # create a subset about 50% of all features
    # first we take randomly indeces from the set
    indeces = np.random.randint(len(all_features), size=len(all_features)//2) 
    less_features = all_features[indeces]
    # fit the kmean => calculate the centroids
    kmeans.fit(less_features)
    centroids = kmeans.cluster_centers_
    edges = np.arange(1,k+2)
    BOF_tr = np.empty((len(data_train),k))
    indexs= np.array([[]])
    for i in range(len(data_train)):
        # calculate the distance from the means using euclidean distance
        # and take the index of the centroid that correspond to the minimum distance 
        dist = distance.cdist(data_train[i], centroids, 'euclidean') 
        indexs = np.argmin(dist,axis = 1)
        hist,_= np.histogram(indexs,edges)
        BOF_tr[i] = preprocessing.normalize(hist.reshape((len(hist),1)), norm='l2').squeeze()
       
       
    BOF_ts = np.empty((len(data_test),k))
    for i in range(len(data_test)):
        dist = distance.cdist(data_test[i], centroids, 'euclidean')
        indexs = np.argmin(dist,axis = 1)
        hist,_= np.histogram(indexs,edges)
        BOF_ts[i] = preprocessing.normalize(hist.reshape((len(hist),1)), norm='l2').squeeze()
     
    return BOF_tr, BOF_ts

##### Testing Our Bag of Visual Words 

In [5]:
def Our_predict_score(detector_fun,descriptor_fun):
    feats = p3.FeatureExtraction(detector_fun, descriptor_fun)
    accs = []
    for k in range(5):
        # Split into a training set and a test set.
        data_train, label_train, data_test, label_test = p3.createTrainTest(feats, k)
        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = OurBagVW(data_train, data_test)
        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)

    mean_accuracy = 100.0*np.mean(accs)
    return mean_accuracy


#### For SURF descriptor and Corner Detector

In [26]:
from timeit import default_timer as timer
start = timer()
accuracyHSS = 0
accuracyHSS = Our_predict_score(detector_fun=detect_fun[0][0], descriptor_fun=desc_fun[0][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detect_fun[0][1],desc_fun[0][1],accuracyHSS))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 48.430
Mean accurancy for Corner Detector with SURF descriptor: 53.241%
Total time: 196.43861900000047


#### Χρησιμοποιώντας τον HOG περιγραφητή και για ανιχνευτή τον Harris-Stephens λαμβάνουμε:

In [27]:
start = timer()
accuracyHSH = 0
accuracyHSH = Our_predict_score(detector_fun=detect_fun[0][0], descriptor_fun=desc_fun[1][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detect_fun[0][1],desc_fun[1][1],accuracyHSH))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 67.310
Mean accurancy for Corner Detector with HOG descriptor: 58.897%
Total time: 193.40779480000037


#### Χρησιμοποιώντας τον SURF περιγραφητή και για ανιχνευτή τον Harris-Laplace λαμβάνουμε:

In [28]:
start = timer()
accuracyHLS = 0
accuracyHLS = Our_predict_score(detector_fun=detect_fun[1][0], descriptor_fun=desc_fun[0][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detect_fun[1][1],desc_fun[0][1],accuracyHLS))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 738.174
Mean accurancy for Multi-Corner Detector with SURF descriptor: 52.552%
Total time: 927.4452167


#### Χρησιμοποιώντας το HOG περιγραφητή και ανιχνευτή το Harris-Laplace τα αποτελέσματα που λαμβάνουμε είναι:

In [29]:
start = timer()
accuracyHLH = 0
accuracyHLH = Our_predict_score(detector_fun=detect_fun[1][0], descriptor_fun=desc_fun[1][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detect_fun[1][1],desc_fun[1][1],accuracyHLH))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 912.083
Mean accurancy for Multi-Corner Detector with HOG descriptor: 54.345%
Total time: 1055.2161811999995


#### Κατηγοροποίηση εικόνων χρησιμποιώντας ως Blobs ανιχνευτές
Χρησιμοποιώντας το SURF περιγραφητή και ανιχνευτή τα Blobs λαμβάνουμε:

In [30]:
start = timer()
accuracyBS = 0
accuracyBS = Our_predict_score(detector_fun=detector_funcsMB[0][0], descriptor_fun=desc_fun[0][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detector_funcsMB[0][1],desc_fun[0][1],accuracyBS))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 31.418
Mean accurancy for Blob Detector with SURF descriptor: 52.000%
Total time: 521.9644865999999


#### Χρησιμοποιώντας το HOG περιγραφητή και ανιχνευτή τα Blobs λαμβάνουμε:


In [31]:
start = timer()
accuracyBH = 0
accuracyBH = Our_predict_score(detector_fun=detector_funcsMB[0][0], descriptor_fun=desc_fun[1][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detector_funcsMB[0][1],desc_fun[1][1],accuracyBH))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 78.816
Mean accurancy for Blob Detector with HOG descriptor: 60.552%
Total time: 525.8917774000001


#### Χρησιμοποιώντας το SURF περιγραφητή και τον πολυκλιμακωτό ανιχνευτή Blobs λαμβάνουμε:

In [32]:
start = timer()
accuracyMBS = 0
accuracyMBS = Our_predict_score(detector_fun=detector_funcsMB[1][0], descriptor_fun=desc_fun[0][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detector_funcsMB[1][1],desc_fun[0][1],accuracyMBS))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 346.240
Mean accurancy for Multi Blob Detector with SURF descriptor: 56.138%
Total time: 765.6960308999987


#### Χρησιμοποιώντας το HOG περιγραφητή και τον πολυκλιμακωτό ανιχνευτή Blobs λαμβάνουμε:

In [33]:
start = timer()
accuracyMBH = 0
accuracyMBH = Our_predict_score(detector_fun=detector_funcsMB[1][0], descriptor_fun=desc_fun[1][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detector_funcsMB[1][1],desc_fun[1][1],accuracyMBH))
   
print('Total time: {}'.format((end-start)))

Time for feature extraction: 395.643
Mean accurancy for Multi Blob Detector with HOG descriptor: 63.034%
Total time: 768.1965318000002


#### Χρησιμοποιώντας το SURF περιγραφητή και τον πολυκλιμακωτό ανιχνευτή Blobs με Box φίλτρα λαμβάνουμε:

In [7]:
start = timer()
accuracyΒBS = 0
accuracyΒBS = Our_predict_score(detector_fun=detector_funcsMB[2][0], descriptor_fun=desc_fun[0][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detector_funcsMB[2][1],desc_fun[0][1],accuracyΒBS))  
print('Total time: {}'.format((end-start)))


Time for feature extraction: 834.163
Mean accurancy for Multi Blob Detector with HOG descriptor: 64.135%
Total time: 987.933562477865


#### Χρησιμοποιώντας το HOG περιγραφητή και τον πολυκλιμακωτό ανιχνευτή Blobs με Box φίλτρα λαμβάνουμε:

In [8]:
start = timer()
accuracyΒBH = 0
accuracyΒBH = Our_predict_score(detector_fun=detector_funcsMB[2][0], descriptor_fun=desc_fun[1][0])
end = timer()                                                           
print('Mean accurancy for {} with {} descriptor: {:.3f}%'.format(detector_funcsMB[2][1],desc_fun[1][1],accuracyΒBH)) 
print('Total time: {}'.format((end-start)))

Time for feature extraction: 952.416
Mean accurancy for Multi Blob Detector with HOG descriptor: 65.672%
Total time: 1136.004635299815


#### Παρουσίαση των αποτελεσμάτων σε ένα κοινό πίνακα χρησιμοποιώντας τη βιβλιοθήκη pandas

In [4]:
!pip install pandas
import pandas as pd


In [10]:
data = [['Harris-Stephens',accuracyHSS,accuracyHSH],
        ['Harris-Laplace',accuracyHLS,accuracyHLH],
        ['Blobs',accuracyBS,accuracyBH],
        ['Multi-Blobs',accuracyMBS,accuracyMBH],
        ['Multi-Box Blobs',accuracyBBS,accuracyBBH]]
df = pd.DataFrame(data,columns=['Detectors','SURF Accuracy %','HOG Accuracy %'])
print(df)

         Detectors  SURF Accuracy %  HOG Accuracy %
0  Harris-Stephens           53.241          58.897
1   Harris-Laplace           52.552          54.345
2            Blobs           52.000          60.552
3      Multi-Blobs           56.138          63.034
4  Multi-Box Blobs           64.135          65.672
